Start!

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
# import time

In [3]:
import astropy.units as u
import numpy as np
import pandas as pd
import astropy.constants as c

In [4]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj
from dolan_junction import DolanJunction as junction
import Default_params as dp


In [5]:
# from qiskit_metal import designs, draw
# from qiskit_metal import MetalGUI, Dict, Headings

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
# from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [6]:
# import seaborn as sns

In [7]:
# phi0 = c.h/2/c.e.si
# T = 30*u.mK
# epsilon_r = 11.45
# cc = 1664080*u.fF/u.m

In [5]:
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
gui = MetalGUI(dp.design)

Component-specific variables

In [6]:
design = dp.design

junction_layer = 5
qubit_layer = 2

## Single Pad--Verify Design

### Draw the single pad transmon

In [7]:
design.delete_all_components()

In [8]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **dp.qb_options)

q1 = transmon(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [9]:
q1.options['junction'] = 'True'

In [10]:
gui.rebuild()

In [11]:
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'
gds = design.renderers.gds

In [12]:
guesses = pd.read_csv(r'data/educated_guess_0403.csv')

In [13]:
options = Dict(taper_len='0.5um',jj_gap = '0.14um')
d = junction(design, 'd', options = options)


In [14]:
k = 3

q1.options['pad_height'] = '{}um'.format(guesses['Sizes (um)'].values[k])
q1.options['pad_width'] = '{}um'.format(guesses['Sizes (um)'].values[k])

q1.options['pocket_width'] = '{}um'.format(guesses['Sizes (um)'].values[k]+2*gap.to(u.um).value)

q1.options['connection_pads']['a']['pad_width'] = guesses['Coupling_len'][k]
q1.options['connection_pads']['a']['pad_height'] = '30um-{}um'.format(guesses['Coupling_gap(um)'][k])
q1.options['connection_pads']['a']['pad_gap'] = '{}um'.format(guesses['Coupling_gap(um)'][k])

d.options['Lj'] = '{}'.format(guesses['Ljs (nH)'][k])

q1.options.layer = qubit_layer
d.options.layer = junction_layer

gui.rebuild()
gui.zoom_on_components(['Q1'])
    

In [15]:
q1, d = jj.place_junction(q1,d)
gui.rebuild()


In [16]:
dp.TQ_options['down_length'] = '40 um'
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.1 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **dp.TQ_options))
TQ1.options['open_termination'] = False

gui.rebuild()
gui.autoscale()

In [17]:
q1,j, cpw, TQ1, design,gui = dp.construct_cpw_qubit(q1,d, TQ1, 5,gui,design,sim =False)
q1, d = jj.place_junction(q1,d)
gui.rebuild()
gui.autoscale()

04:38PM 11s INFO [delete_component]: Called delete_component cpw_1, but such a component is not in the design cache dictionary of components.


214.0 um
214.0 um
-(214.0 um/2+40um-80 um/4)+0


04:38PM 12s INFO [delete_component]: Called delete_component cpw_1, but such a component is not in the design cache dictionary of components.


In [29]:
import geopandas as gpd

In [49]:
cpw

name:    cpw_1
class:   RouteMixed            
options: 
  'chip'              : 'main',                       
  'layer'             : 2,                            
  'pin_inputs'        : {
       'start_pin'         : {
            'component'         : 'Q1',                         
            'pin'               : 'a',                          
                             },
       'end_pin'           : {
            'component'         : 'TQ1',                        
            'pin'               : 'second_end',                 
                             },
                        },
  'fillet'            : '30 um',                      
  'lead'              : {
       'start_straight'    : '20um',                       
       'end_straight'      : '50um',                       
       'start_jogged_extension': '',                           
       'end_jogged_extension': '',                           
                        },
  'total_length'      : '5 mm',         

In [30]:
path = cpw.qgeometry_table('path')['geometry'][4]

In [60]:
large = path.offset_curve(0.001,join_style=2,miter_limit=2.0)

AttributeError: 'LineString' object has no attribute 'offset_curve'

In [67]:
cpw.options.trace_width

'12.4 um'

In [62]:
coordinates = path.coords[:]

In [65]:
design.parse_value(cpw.options.fillet)

0.03

In [69]:
corrected = []
r = design.parse_value(cpw.options.fillet)
center_pin = design.parse_value(cpw.options.trace_width)
gap_w = design.parse_value(cpw.options.trace_gap)
r_in = r-center_pin/2-gap_w
# for i in coordinates:

In [73]:
def find_line_direction(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    if x1 == x2:
        if y1>y2:
            return 270
        elif y1<y2:
            return 90
        else:
            return 0
    elif y1 == y2:
        if x1>x2:
            return 180
        elif x1<x2:
            return 0
        else: return 0
    else:
        return ((np.arctan((y2-y1)/(x2-x1))*180/np.pi)+360)%360

In [72]:
for i, cords in enumerate(coordinates):
    final_cords = []
    if i ==0:
        final_cords += cords
        continue
    elif i == len(coordinates)-1:
        final_cords += cords
        break
    angle = find_line_direction(coordinates[i-1],cords)
    next_angle = find_line_direction(cords,coordinates[i+1])
    if angle == next_angle:
        continue
    if angle == 
    

1
2
3
4
5
6
7
8
9
10
11


In [71]:
r

0.03

In [48]:
'mix' in str(type(cpw))

True

In [18]:
q1.options.layer = qubit_layer
cpw.options.layer = qubit_layer
TQ1.options.layer = qubit_layer
j.options.layer = junction_layer
gui.rebuild()
gui.autoscale()

In [22]:
design.chips.main.size['size_x'] = '10mm'
design.chips.main.size['size_y'] = '10mm'
gui.rebuild()
a_gds = design.renderers.gds

In [39]:
a_gds.options.cheese.edge_nocheese = '300um'
a_gds.options.no_cheese.buffer = '50um'

In [27]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True

In [40]:
a_gds.export_to_gds('export_test.gds')

08:15PM 18s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
08:15PM 18s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"/Users/wendy/Desktop/resources".


1

In [30]:
a_gds = design.renderers.gds

In [157]:
a_gds.options.tolerance = '0.00001'
a_gds.options.cheese.edge_nocheese = '300um'
a_gds.options.no_cheese.buffer = '40um'

In [224]:
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': True,
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': 'Customized_Component/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: False}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '0um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: False}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2',
 'ground_main_1': {'view_in_file': {'main': {1: True}}}}

In [208]:
a_gds.options['path_filename'] = 'Customized_Component/Fake_Junctions.GDS'

In [51]:
a_gds.options.no_cheese.buffer = '0um'

a_gds.options['cheese']['view_in_file']['main'][5] = False
a_gds.options['no_cheese']['view_in_file']['main'][5] = False
a_gds.options['ground_plane'] = True

In [49]:
q1.options.layer = 1
d.options.layer = 5
gui.rebuild()

0.002
10.0 nH 0.1 uA / um2 0.00024198969005548028


In [25]:
a_gds.options['ground_plane'] = True

In [52]:

# a_gds.options['short_segments_to_not_fillet'] = True
# a_gds.options['check_short_segments_by_scaling_fillet'] = 1.0
a_gds.export_to_gds('Dolan.gds')#, highlight_qcomponents=['TQ1'])

03:53PM 34s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
03:53PM 34s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"/Users/wendy/Desktop/resources".


1

In [32]:
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': True,
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [127]:
design.components.keys()

['d']